# **Tutoriel** - Fine-tuning de Llama2

Ce carnet de code permet de fine-tuner Llama 2 pour générer du texte en français écrit du 17e siècle. Le fine-tuning est basé sur un corpus de 2000 instructions extraites de romans français publiés entre 1600 et 1700. C'est un bon exemple pour mesurer les capacités du fine-tuning (et accessoirement faire mieux que ChatGPT dans ce domaine précis).

Vous pouvez également utiliser ce carnet de code à partir de n'importe quel jeu de données hébergé sur HuggingFace sous réserve d'avoir la structure ###human / ###assistant (et non pas le format classique de Alpaca : instruction/input/output)

# Installation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/llama"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/llama


Nous allons utilisé un petit script en Python créé par Younes Belkada.

In [ ]:
!rm -rf e90381ed142ee80c8e7ea602b18d50f0
!git clone https://gist.github.com/Pclanglais/e90381ed142ee80c8e7ea602b18d50f0

Cloning into 'e90381ed142ee80c8e7ea602b18d50f0'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 38 (delta 14), reused 10 (delta 4), pack-reused 0
Unpacking objects: 100% (38/38), 6.25 KiB | 20.00 KiB/s, done.


We chargeons les extensions nécessaires avec la bonne version

In [ ]:
!pip install accelerate==0.21.0
!pip install peft==0.4.0
!pip install bitsandbytes==0.40.2
!pip install transformers==4.30.2
!pip install trl==0.4.7

Nous allons dans le dossier contenant le script (désolé pour le nom imprononceable)

In [ ]:
%cd "e90381ed142ee80c8e7ea602b18d50f0"

/content/drive/MyDrive/llama/e90381ed142ee80c8e7ea602b18d50f0


Nous lançons le script. Vous pouvez ajuster le nombre de steps pour avoir un entraînement plus intensif. En l'état actuel, l'entraînement prendra une quinzaine de minutes sur la version pro de Google Colab.

In [ ]:
!python finetune_llama_v2.py --dataset_name "AlexanderDoria/novel17_test" --max_steps 500 --merge_and_push True --model_name "daryl149/llama-2-7b-chat-hf"

2023-07-19 13:28:29.325023: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100% 2/2 [00:14<00:00,  7.12s/it]
Found cached dataset json (/root/.cache/huggingface/datasets/AlexanderDoria___json/AlexanderDoria--novel17_test-5d434af8006df014/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
Loading cached processed dataset at /root/.cache/huggingface/datasets/AlexanderDoria___json/AlexanderDoria--novel17_test-5d434af8006df014/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-2fc5e2a3395f2e8d.arrow
  0% 0/500 [00:00<?, ?it/s]You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is fast

## Inférence

Maintenant que l'entraînement est fini, nous allons générer du texte (c'est l'inférence). À noter que cela prend beaucoup de mémoire pour l'instant : je recommande de réinitialiser le runtime de la session sur colab.

Nous chargeons le modèle

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained("results/final_merged_checkpoint")

Nous générons maintenant le texte. Comme le modèle n'a pas été beaucoup entraîné, nous lui donnons un début de réponse : cela aide le modèle à se focaliser. N'hésitez pas à changer le max_length pour avoir des réponses plus longues (c'est le nombre maximal de tokens).

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("daryl149/llama-2-7b-chat-hf")

from transformers import pipeline

generator = pipeline(task="text-generation", model=model, tokenizer = tokenizer, max_length=200)

result = generator("Ecrivez un texte en ancien français du 17e siècle sur le meilleur moyen de voyager sur la lune ### Assistant: Je serois parti sur la Lune")

print(result)

[{'generated_text': "Ecrivez un texte en ancien français du 17e siècle sur le meilleur moyen de voyager sur la lune ### Assistant: Je serois parti sur la Lune, mais comment fuste bon le plus beau moyen de faire? Quoi! Sçavans le plus savant d'autrefois dit que les anciens de la lune étoient des esprits sublimes et de bon sens, qui ne savoient que faire des corps mortels qui les entornoient, et que donc le plus beau moyen de voyager sur la lune seroit de ne point faire de démonstres et de ne point savoier les choses de ce monde-ci, mais de méditer sur les choses éternelles et de ne point songer à celles qui sont de ce monde.Ћommo d'autrefois disoit que les Bretons avoit beau voyage"}]
